## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [1]:
from importlib import reload

import numpy as np

from DDQN.DDQN import DDQNAgent
import hockey.hockey_env as h_env

In [2]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

### Normal Gameplay

#### Weak Opponent

In [3]:
reload(h_env)
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)

env.reset()

/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/kivanc/miniconda3/envs/rl/lib/python3.11/site-packages/gymnasium/envs/registration.py:642: UserWarning: WARN: Overriding environment Hockey-One-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.21159029, -0.04174399,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [ ]:
STATE_PATH = "./DDQN/models/hockey_normal_dev/"

agent1 = DDQNAgent(
    env.observation_space,
    env.discrete_action_space
)
agent1.load_state(STATE_PATH)

agent2 = h_env.BasicOpponent(weak=True)

/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path.join(load_dir, "Q_model.ckpt"))


In [ ]:
STEPS = 1000000

obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_agent2 = env.obs_agent_two()

for i in range(STEPS):
    env.render()
    a1 = agent1.act(obs)
    a1 = env.discrete_to_continous_action(a1)
    a2 = agent2.act(obs_agent2)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_agent2 = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {i} steps")
        break

Episode done in 47 steps


In [16]:
env.close()